###  The following tasks come from one of the Udemy courses where they were completed using Python pandas. I've also done them using Postgresql.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [17]:
# importing the data frame
df=pd.read_csv('/movies_complete.csv', parse_dates=True)

In [18]:
# adding column profit = revenue_musd- budget_musd
df['profit']=df['revenue_musd'].sub(df['budget_musd'])

In [19]:
# adding column ROI = revenue_musd/budget_musd
df['roi']=df['revenue_musd'].div(df['budget_musd'])

In [20]:
df.columns

Index(['id', 'title', 'tagline', 'release_date', 'genres',
       'belongs_to_collection', 'original_language', 'budget_musd',
       'revenue_musd', 'production_companies', 'production_countries',
       'vote_count', 'vote_average', 'popularity', 'runtime', 'overview',
       'spoken_languages', 'poster_path', 'cast', 'cast_size', 'crew_size',
       'director', 'profit', 'roi'],
      dtype='object')

In [21]:
# selecting columns
df=df[['id', 'title', 'release_date', 'genres','belongs_to_collection','budget_musd',
       'revenue_musd', 'production_companies','vote_count', 'vote_average', 'popularity', 
       'cast', 'cast_size', 'crew_size',
       'director','profit', 'roi' ]]
df.columns=['id', 'title', 'release_date', 'genres','belongs_to_collection','budget_musd',
       'revenue_musd', 'production_companies','vote_count', 'vote_average', 'popularity', 
       'casting', 'cast_size', 'crew_size',
       'director','profit', 'roi']

In [22]:
df.head(5)

,id,title,release_date,genres,belongs_to_collection,budget_musd,revenue_musd,production_companies,vote_count,vote_average,popularity,casting,cast_size,crew_size,director,profit,roi
0,862,Toy Story,1995-10-30,Animation|Comedy|Family,Toy Story Collection,30.0,373.554033,Pixar Animation Studios,5415.0,7.7,21.946943,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,13,106,John Lasseter,343.554033,12.451801
1,8844,Jumanji,1995-12-15,Adventure|Fantasy|Family,NaN,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,2413.0,6.9,17.015539,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,26,16,Joe Johnston,197.797249,4.043035
2,15602,Grumpier Old Men,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,NaN,NaN,Warner Bros.|Lancaster Gate,92.0,6.5,11.712900,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,7,4,Howard Deutch,NaN,NaN
3,31357,Waiting to Exhale,1995-12-22,Comedy|Drama|Romance,NaN,16.0,81.452156,Twentieth Century Fox Film Corporation,34.0,6.1,3.859495,Whitney Houston|Angela Bassett|Loretta Devine|...,10,10,Forest Whitaker,65.452156,5.090760
4,11862,Father of the Bride Part II,1995-02-10,Comedy,Father of the Bride Collection,NaN,76.578911,Sandollar Productions|Touchstone Pictures,173.0,5.7,8.387519,Steve Martin|Diane Keaton|Martin Short|Kimberl...,12,7,Charles Shyer,NaN,NaN


### Importing data into Postgresql using Pythona

In [2]:
engine = create_engine('postgresql://kinga@localhost:5432/movies')
from sqlalchemy.types import Numeric

In [31]:
df.to_sql('movies', engine, if_exists='replace', dtype={'budget_musd':Numeric(),'revenue_musd':Numeric(),
        'vote_count':Numeric(), 'vote_average':Numeric(), 'popularity':Numeric(),
        'profit':Numeric(), 'roi':Numeric()}, index=False)

In [32]:
# checking if the dataset has been correctly loaded
movies=""" SELECT * FROM movies
            FETCH FIRST 5 ROWS ONLY"""
movies=pd.read_sql(movies, engine)
movies

,id,title,release_date,genres,belongs_to_collection,budget_musd,revenue_musd,production_companies,vote_count,vote_average,popularity,casting,cast_size,crew_size,director,profit,roi
0,862,Toy Story,1995-10-30,Animation|Comedy|Family,Toy Story Collection,30.0,373.554033,Pixar Animation Studios,5415.0,7.7,21.946943,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,13,106,John Lasseter,343.554033,12.451801
1,8844,Jumanji,1995-12-15,Adventure|Fantasy|Family,None,65.0,262.797249,TriStar Pictures|Teitler Film|Interscope Commu...,2413.0,6.9,17.015539,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,26,16,Joe Johnston,197.797249,4.043035
2,15602,Grumpier Old Men,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,NaN,NaN,Warner Bros.|Lancaster Gate,92.0,6.5,11.712900,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,7,4,Howard Deutch,NaN,NaN
3,31357,Waiting to Exhale,1995-12-22,Comedy|Drama|Romance,None,16.0,81.452156,Twentieth Century Fox Film Corporation,34.0,6.1,3.859495,Whitney Houston|Angela Bassett|Loretta Devine|...,10,10,Forest Whitaker,65.452156,5.090760
4,11862,Father of the Bride Part II,1995-02-10,Comedy,Father of the Bride Collection,NaN,76.578911,Sandollar Productions|Touchstone Pictures,173.0,5.7,8.387519,Steve Martin|Diane Keaton|Martin Short|Kimberl...,12,7,Charles Shyer,NaN,NaN


## Data analysis

#### 1. Science Fiction Action Movie with Bruce Willis (high Rating)

In [33]:
Bruce_Willis="""SELECT title, release_date, genres, casting, vote_average, vote_count FROM movies
                WHERE (genres ILIKE '%Science Fiction%' OR genres ILIKE '%Action%')
                AND casting ILIKE '%Bruce Willis%'
                ORDER BY vote_average DESC"""
Bruce_Willis=pd.read_sql(Bruce_Willis, engine)
Bruce_Willis.head(5)

,title,release_date,genres,casting,vote_average,vote_count
0,Die Hard,1988-07-15,Action|Thriller,Bruce Willis|Alan Rickman|Alexander Godunov|Bo...,7.5,4005.0
1,Twelve Monkeys,1995-12-29,Science Fiction|Thriller|Mystery,Bruce Willis|Madeleine Stowe|Brad Pitt|Christo...,7.4,2470.0
2,The Fifth Element,1997-05-07,Adventure|Fantasy|Action|Thriller|Science Fiction,Bruce Willis|Gary Oldman|Ian Holm|Milla Jovovi...,7.3,3962.0
3,Sin City,2005-04-01,Action|Thriller|Crime,Jessica Alba|Devon Aoki|Clive Owen|Rosario Daw...,7.2,2755.0
4,Unbreakable,2000-11-13,Science Fiction|Thriller|Drama,Bruce Willis|Samuel L. Jackson|Robin Wright|Sp...,6.9,1994.0


#### 2. Most Successful Pixar Studio Movies between 2010 and 2015 (high Revenue)

In [34]:
Pixar="""SELECT title, release_date,  production_companies, revenue_musd FROM movies
        WHERE production_companies LIKE '%Pixar%'
        AND release_date BETWEEN '2010-01-01' AND '2015-12-31'
        AND revenue_musd IS NOT NULL
        ORDER BY revenue_musd DESC"""
Pixar=pd.read_sql(Pixar, engine)
Pixar

,title,release_date,production_companies,revenue_musd
0,Toy Story 3,2010-06-16,Walt Disney Pictures|Pixar Animation Studios,1066.969703
1,Inside Out,2015-06-09,Walt Disney Pictures|Pixar Animation Studios,857.611174
2,Monsters University,2013-06-20,Walt Disney Pictures|Pixar Animation Studios,743.559607
3,Cars 2,2011-06-11,Walt Disney Pictures|Pixar Animation Studios,559.852396
4,Brave,2012-06-21,Walt Disney Pictures|Pixar Animation Studios,538.983207
5,The Good Dinosaur,2015-11-14,Walt Disney Pictures|Pixar Animation Studios,331.926147


#### 3. Are Franchises more successful?
- comparing average revenue, profit, roi, popuarity of franchises and stand alone movies 

In [35]:
franchise_vs_standalone="""WITH cte as(
                        SELECT title, belongs_to_collection, revenue_musd, profit, roi, popularity,
                        CASE WHEN belongs_to_collection IS NOT NULL THEN 'franchise' 
                        ELSE 'stand alone' END AS franchise
                        FROM movies)
                        SELECT franchise,  round(avg(revenue_musd),2) as avg_revenue, 
                        round(avg(profit),2) as avg_profit, round(avg(roi), 2) as avg_roi,
                         round(avg(popularity),2) as avg_popularity
                        FROM cte
                        GROUP BY franchise"""
franchise_vs_standalone=pd.read_sql(franchise_vs_standalone, engine)
franchise_vs_standalone

,franchise,avg_revenue,avg_profit,avg_roi,avg_popularity
0,stand alone,44.74,32.95,6966.39,2.59
1,franchise,165.71,148.88,856.40,6.25


#### 4.  Find the most successful Franchises in terms of

- total number of movies
- total & mean budget
- total & mean revenue
- mean rating
- Show only data without null values

In [7]:
most_successful="""WITH most_successful as 
                        (SELECT belongs_to_collection, count(title) as number_of_movies, 
                        round(sum(budget_musd),2) as total_budget, round(avg(budget_musd),2) as avg_budget, 
                        round(sum(revenue_musd),2) as total_revenue, round(avg(revenue_musd),2) as avg_revenue,
                        round(avg(vote_average),2) as avg_rating
                        FROM movies 
                        GROUP BY belongs_to_collection
                        ORDER by number_of_movies DESC) 
                    SELECT * FROM most_successful
                    WHERE most_successful IS NOT NULL"""
most_successful=pd.read_sql(most_successful, engine)
most_successful.head(10)

,belongs_to_collection,number_of_movies,total_budget,avg_budget,total_revenue,avg_revenue,avg_rating
0,James Bond Collection,26,1539.65,59.22,7106.97,273.35,6.34
1,Pokémon Collection,20,250.72,50.14,601.87,66.87,6.06
2,Dragon Ball Z (Movie) Collection,15,5.00,5.00,112.12,56.06,6.61
3,The Land Before Time Collection,14,12.30,12.30,84.46,84.46,6.16
4,The Amityville Horror Collection,12,21.02,5.26,213.35,53.34,4.36
5,Friday the 13th Collection,12,84.40,7.03,464.90,38.74,5.43
6,Hellraiser Collection,10,27.35,3.04,55.86,13.97,4.99
7,Dracula (Hammer Series),9,0.98,0.33,0.91,0.45,6.11
8,Children of the Corn Collection,9,8.70,2.18,20.98,10.49,4.30
9,Ernest Collection,9,9.00,4.50,51.71,25.86,4.51
